In [1159]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [1278]:
from sklearn import svm, tree, linear_model, neighbors, naive_bayes, ensemble, discriminant_analysis, gaussian_process
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score, GridSearchCV, train_test_split
from sklearn.linear_model import LogisticRegressionCV
from sklearn.feature_selection import RFECV
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler
from sklearn import feature_selection
from sklearn import metrics
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB


## Load match related data

In [1280]:

#df_2014 = pd.read_csv("../data/afl_results_2014.csv")
#print(df_2014.shape)

#df_2015 = pd.read_csv("../data/afl_results_2015.csv")
#print(df_2015.shape)

#df_2016 = pd.read_csv("../data/afl_results_2016.csv")
#print(df_2016.shape)

df_2017 = pd.read_csv("../data/afl_results_2017.csv")
print(df_2017.shape)

df_2018 = pd.read_csv("../data/afl_results_2018.csv")
print(df_2018.shape)

df_2019 = pd.read_csv("../data/afl_results_2019.csv")
print(df_2019.shape)
df_2020 = pd.read_csv("../data/afl_results_2020.csv")
print(df_2020.shape)
df_2021 = pd.read_csv("../data/afl_results_2021.csv")
print(df_2021.shape)
df_2022 = pd.read_csv("../data/afl_results_2022.csv")
print(df_2022.shape)

df_all = pd.concat([df_2017, df_2018, df_2019, df_2020, df_2021,df_2022], axis=0)
print(df_all.shape)
df_all.columns

(207, 75)
(207, 75)
(207, 75)
(162, 75)
(207, 75)
(82, 75)
(1072, 75)


Index(['match.name', 'match.date', 'match.status', 'match.matchId',
       'match.venue', 'match.utcStartTime', 'match.homeTeamId',
       'match.awayTeamId', 'match.round', 'match.venueLocalStartTime',
       'match.abbr', 'match.twitterHashTag', 'match.homeTeam.name',
       'match.homeTeam.timeZone', 'match.homeTeam.teamId',
       'match.homeTeam.abbr', 'match.homeTeam.nickname', 'match.awayTeam.name',
       'match.awayTeam.timeZone', 'match.awayTeam.teamId',
       'match.awayTeam.abbr', 'match.awayTeam.nickname', 'venue.address',
       'venue.name', 'venue.state', 'venue.timeZone', 'venue.venueId',
       'venue.abbreviation', 'venue.capacity', 'venue.groundDimension',
       'venue.latitude', 'venue.longitude', 'round.name', 'round.year',
       'round.roundId', 'round.abbreviation', 'round.competitionId',
       'round.roundNumber', 'status', 'matchId', 'scoreWorm', 'scoreMap',
       'lastUpdated', 'homeTeamScore.periodScore',
       'homeTeamScore.rushedBehinds', 'homeTeamS

In [1281]:
df_all.head()

,match.name,match.date,match.status,match.matchId,match.venue,match.utcStartTime,match.homeTeamId,match.awayTeamId,match.round,match.venueLocalStartTime,...,homeTeamScoreChart.rightPosters,homeTeamScoreChart.rushedBehinds,homeTeamScoreChart.touchedBehinds,awayTeamScoreChart.goals,awayTeamScoreChart.leftBehinds,awayTeamScoreChart.rightBehinds,awayTeamScoreChart.leftPosters,awayTeamScoreChart.rightPosters,awayTeamScoreChart.rushedBehinds,awayTeamScoreChart.touchedBehinds
0,Carlton Vs Richmond,2017-03-23 08:20:00,CONCLUDED,CD_M20170140101,CD_V40,2017-03-23T08:20:00,CD_T30,CD_T120,CD_R201701401,2017-03-23T19:20:00,...,0,1,0,20,5,3,0,1,3,0
1,Collingwood Vs Western Bulldogs,2017-03-24 08:50:00,CONCLUDED,CD_M20170140102,CD_V40,2017-03-24T08:50:00,CD_T40,CD_T140,CD_R201701401,2017-03-24T19:50:00,...,1,3,0,15,3,2,0,1,4,0
2,St Kilda Vs Melbourne,2017-03-25 05:35:00,CONCLUDED,CD_M20170140103,CD_V190,2017-03-25T05:35:00,CD_T130,CD_T90,CD_R201701401,2017-03-25T16:35:00,...,0,2,0,18,4,5,0,1,2,0
3,Sydney Swans Vs Port Adelaide,2017-03-25 05:35:00,CONCLUDED,CD_M20170140104,CD_V60,2017-03-25T05:35:00,CD_T160,CD_T110,CD_R201701401,2017-03-25T16:35:00,...,0,1,0,17,1,6,0,0,0,1
4,Essendon Vs Hawthorn,2017-03-25 08:25:00,CONCLUDED,CD_M20170140106,CD_V40,2017-03-25T08:25:00,CD_T50,CD_T80,CD_R201701401,2017-03-25T19:25:00,...,0,2,0,12,5,9,0,0,5,0


In [1282]:

df_all['Date'] = pd.to_datetime(df_all['match.date']).dt.strftime("%Y-%m-%d")
df_all[['Date','match.date']].head()

,Date,match.date
0,2017-03-23,2017-03-23 08:20:00
1,2017-03-24,2017-03-24 08:50:00
2,2017-03-25,2017-03-25 05:35:00
3,2017-03-25,2017-03-25 05:35:00
4,2017-03-25,2017-03-25 08:25:00


In [1283]:
# Read player data

df_players_2017 = pd.read_csv("../data/afl_players_stats_2017.csv")
print(df_players_2017.shape)
df_players_2018 = pd.read_csv("../data/afl_players_stats_2018.csv")
print(df_players_2018.shape)
df_players_2019 = pd.read_csv("../data/afl_players_stats_2019.csv")
print(df_players_2019.shape)
df_players_2020 = pd.read_csv("../data/afl_players_stats_2020.csv")
print(df_players_2020.shape)
df_players_2021 = pd.read_csv("../data/afl_players_stats_2021.csv")
print(df_players_2021.shape)
df_players_2022 = pd.read_csv("../data/afl_players_stats_2022.csv")
print(df_players_2022.shape)

df_players = pd.concat([df_players_2017, df_players_2018, df_players_2019,df_players_2020,df_players_2021,df_players_2022], axis=0)
print(df_players.shape)
df_players.columns


(9108, 59)
(9108, 59)
(9108, 59)
(7128, 59)
(9526, 59)
(3726, 59)
(47704, 59)


Index(['Season', 'Round', 'Date', 'Local.start.time', 'Venue', 'Attendance',
       'Home.team', 'HQ1G', 'HQ1B', 'HQ2G', 'HQ2B', 'HQ3G', 'HQ3B', 'HQ4G',
       'HQ4B', 'Home.score', 'Away.team', 'AQ1G', 'AQ1B', 'AQ2G', 'AQ2B',
       'AQ3G', 'AQ3B', 'AQ4G', 'AQ4B', 'Away.score', 'First.name', 'Surname',
       'ID', 'Jumper.No.', 'Playing.for', 'Kicks', 'Marks', 'Handballs',
       'Goals', 'Behinds', 'Hit.Outs', 'Tackles', 'Rebounds', 'Inside.50s',
       'Clearances', 'Clangers', 'Frees.For', 'Frees.Against',
       'Brownlow.Votes', 'Contested.Possessions', 'Uncontested.Possessions',
       'Contested.Marks', 'Marks.Inside.50', 'One.Percenters', 'Bounces',
       'Goal.Assists', 'Time.on.Ground..', 'Substitute', 'Umpire.1',
       'Umpire.2', 'Umpire.3', 'Umpire.4', 'group_id'],
      dtype='object')

In [1284]:
df_players.head()

,Season,Round,Date,Local.start.time,Venue,Attendance,Home.team,HQ1G,HQ1B,HQ2G,...,One.Percenters,Bounces,Goal.Assists,Time.on.Ground..,Substitute,Umpire.1,Umpire.2,Umpire.3,Umpire.4,group_id
0,2017,1,2017-03-23,1920,M.C.G.,73137,Carlton,4,3,7,...,0,0,0,82,NaN,Sam Hay,Mathew Nicholls,Matt Stevic,NaN,NaN
1,2017,1,2017-03-23,1920,M.C.G.,73137,Carlton,4,3,7,...,1,0,0,81,NaN,Sam Hay,Mathew Nicholls,Matt Stevic,NaN,NaN
2,2017,1,2017-03-23,1920,M.C.G.,73137,Carlton,4,3,7,...,0,0,0,84,NaN,Sam Hay,Mathew Nicholls,Matt Stevic,NaN,NaN
3,2017,1,2017-03-23,1920,M.C.G.,73137,Carlton,4,3,7,...,1,0,0,76,NaN,Sam Hay,Mathew Nicholls,Matt Stevic,NaN,NaN
4,2017,1,2017-03-23,1920,M.C.G.,73137,Carlton,4,3,7,...,0,2,0,86,NaN,Sam Hay,Mathew Nicholls,Matt Stevic,NaN,NaN


In [1285]:
def get_aggregate_player_stats(df=None):

    agg_stats = (df.rename(columns={ # Rename columns to lowercase
                    'Home.team': 'match.homeTeam.name',
                    'Away.team': 'match.awayTeam.name',
                    })
                   .groupby(by=['Date', 'Season', 'match.homeTeam.name', 'match.awayTeam.name'], as_index=False) # Groupby to aggregate the stats for each game
                   .sum()
                   #.drop(columns=['DE', 'TOG', 'Match_id']) # Drop columns
                   .assign(date=lambda df: pd.to_datetime(df.Date, format="%Y-%m-%d")) # Create a datetime object
                   .sort_values(by='Date')
                   .reset_index(drop=True))
    return agg_stats

In [1286]:
agg_player = get_aggregate_player_stats(df_players)
print(agg_player.shape)
print(agg_player.columns)

(1071, 50)
Index(['Date', 'Season', 'match.homeTeam.name', 'match.awayTeam.name',
       'Local.start.time', 'Attendance', 'HQ1G', 'HQ1B', 'HQ2G', 'HQ2B',
       'HQ3G', 'HQ3B', 'HQ4G', 'HQ4B', 'Home.score', 'AQ1G', 'AQ1B', 'AQ2G',
       'AQ2B', 'AQ3G', 'AQ3B', 'AQ4G', 'AQ4B', 'Away.score', 'ID', 'Kicks',
       'Marks', 'Handballs', 'Goals', 'Behinds', 'Hit.Outs', 'Tackles',
       'Rebounds', 'Inside.50s', 'Clearances', 'Clangers', 'Frees.For',
       'Frees.Against', 'Brownlow.Votes', 'Contested.Possessions',
       'Uncontested.Possessions', 'Contested.Marks', 'Marks.Inside.50',
       'One.Percenters', 'Bounces', 'Goal.Assists', 'Time.on.Ground..',
       'Substitute', 'group_id', 'date'],
      dtype='object')


In [1287]:
print(df_all.shape)
print(agg_player.shape)

#df_all[['Date', 'match.homeTeam.name', 'match.awayTeam.name']]
#agg_player[['Date', 'match.homeTeam.name', 'match.awayTeam.name']]


(1072, 76)
(1071, 50)


In [1301]:
df_all[['round.name', 'round.year',
       'round.roundId', 'round.abbreviation', 'round.competitionId',
       'round.roundNumber']].tail()

,round.name,round.year,round.roundId,round.abbreviation,round.competitionId,round.roundNumber
77,Round 9,2022,CD_R202201409,Rd 9,CD_S2022014,9
78,Round 9,2022,CD_R202201409,Rd 9,CD_S2022014,9
79,Round 9,2022,CD_R202201409,Rd 9,CD_S2022014,9
80,Round 9,2022,CD_R202201409,Rd 9,CD_S2022014,9
81,Round 10,2022,CD_R202201410,Rd 10,CD_S2022014,10


In [1302]:
afl_df = df_all.merge(agg_player, on=['Date', 'match.homeTeam.name', 'match.awayTeam.name'], how='left')





In [1303]:
afl_df.shape

(1072, 123)

In [1293]:
print(afl_df.columns.tolist())

    


['match.name', 'match.date', 'match.status', 'match.matchId', 'match.venue', 'match.utcStartTime', 'match.homeTeamId', 'match.awayTeamId', 'match.round', 'match.venueLocalStartTime', 'match.abbr', 'match.twitterHashTag', 'match.homeTeam.name', 'match.homeTeam.timeZone', 'match.homeTeam.teamId', 'match.homeTeam.abbr', 'match.homeTeam.nickname', 'match.awayTeam.name', 'match.awayTeam.timeZone', 'match.awayTeam.teamId', 'match.awayTeam.abbr', 'match.awayTeam.nickname', 'venue.address', 'venue.name', 'venue.state', 'venue.timeZone', 'venue.venueId', 'venue.abbreviation', 'venue.capacity', 'venue.groundDimension', 'venue.latitude', 'venue.longitude', 'round.name', 'round.year', 'round.roundId', 'round.abbreviation', 'round.competitionId', 'round.roundNumber', 'status', 'matchId', 'scoreWorm', 'scoreMap', 'lastUpdated', 'homeTeamScore.periodScore', 'homeTeamScore.rushedBehinds', 'homeTeamScore.minutesInFront', 'homeTeamScore.matchScore.totalScore', 'homeTeamScore.matchScore.goals', 'homeTeam

In [1304]:
df_fixture = pd.read_csv("../data/fixture_2022.csv")
print(df_fixture.shape)
df_fixture.columns

df_next_games_teams = df_fixture[(df_fixture['status'] != "CONCLUDED") & (df_fixture['round.roundNumber'] == 11)]
df_next_games_teams = df_next_games_teams[['home.team.name','away.team.name']]
df_next_games_teams = df_next_games_teams.rename(columns={'home.team.name': 'match.homeTeam.name', 'away.team.name': 'match.awayTeam.name'})
df_next_games_teams

(198, 54)


,match.homeTeam.name,match.awayTeam.name
90,Sydney Swans,Richmond
91,Brisbane Lions,GWS Giants
92,Geelong Cats,Adelaide Crows
93,Melbourne,Fremantle
94,West Coast Eagles,Western Bulldogs
95,Gold Coast Suns,Hawthorn
96,St Kilda,North Melbourne
97,Collingwood,Carlton
98,Port Adelaide,Essendon


In [1305]:

df_fixture.columns
print(df_fixture.columns.tolist())



['id', 'providerId', 'utcStartTime', 'status', 'compSeason.id', 'compSeason.providerId', 'compSeason.name', 'compSeason.shortName', 'compSeason.currentRoundNumber', 'round.id', 'round.providerId', 'round.abbreviation', 'round.name', 'round.roundNumber', 'round.byes', 'home.team.id', 'home.team.providerId', 'home.team.name', 'home.team.abbreviation', 'home.team.nickname', 'home.team.teamType', 'home.team.club.id', 'home.team.club.providerId', 'home.team.club.name', 'home.team.club.abbreviation', 'home.team.club.nickname', 'home.score.goals', 'home.score.behinds', 'home.score.totalScore', 'home.score.superGoals', 'away.team.id', 'away.team.providerId', 'away.team.name', 'away.team.abbreviation', 'away.team.nickname', 'away.team.teamType', 'away.team.club.id', 'away.team.club.providerId', 'away.team.club.name', 'away.team.club.abbreviation', 'away.team.club.nickname', 'away.score.goals', 'away.score.behinds', 'away.score.totalScore', 'away.score.superGoals', 'venue.id', 'venue.providerId'

In [1306]:
df_all.shape

(1072, 76)

In [1178]:
df_all.sort_values('match.date', inplace=True)
df_all.reset_index(inplace=True)
df_all.drop('index', axis=1, inplace=True)

In [1179]:
afl_df.tail()

,match.name,match.date,match.status,match.matchId,match.venue,match.utcStartTime,match.homeTeamId,match.awayTeamId,match.round,match.venueLocalStartTime,...,Uncontested.Possessions,Contested.Marks,Marks.Inside.50,One.Percenters,Bounces,Goal.Assists,Time.on.Ground..,Substitute,group_id,date
985,Geelong Cats Vs GWS Giants,2021-09-03 09:50:00,CONCLUDED,CD_M20210142501,CD_V2925,2021-09-03T09:50:00,CD_T70,CD_T1010,CD_R202101425,2021-09-03T17:50:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
986,Brisbane Lions Vs Western Bulldogs,2021-09-04 09:20:00,CONCLUDED,CD_M20210142502,CD_V20,2021-09-04T09:20:00,CD_T20,CD_T140,CD_R202101425,2021-09-04T19:20:00,...,396.0,15.0,18.0,136.0,13.0,17.0,3601.0,0.0,0.0,2021-09-04
987,Melbourne Vs Geelong Cats,2021-09-10 09:50:00,CONCLUDED,CD_M20210142601,CD_V2925,2021-09-10T09:50:00,CD_T90,CD_T70,CD_R202101426,2021-09-10T17:50:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
988,Port Adelaide Vs Western Bulldogs,2021-09-11 09:40:00,CONCLUDED,CD_M20210142602,CD_V6,2021-09-11T09:40:00,CD_T110,CD_T140,CD_R202101426,2021-09-11T19:10:00,...,421.0,29.0,23.0,114.0,13.0,16.0,3599.0,0.0,0.0,2021-09-11
989,Melbourne Vs Western Bulldogs,2021-09-25 09:15:00,CONCLUDED,CD_M20210142701,CD_V2925,2021-09-25T09:15:00,CD_T90,CD_T140,CD_R202101427,2021-09-25T17:15:00,...,440.0,14.0,21.0,99.0,16.0,23.0,3601.0,0.0,0.0,2021-09-25


In [1180]:
#from pandas.plotting import scatter_matrix

In [1181]:
#scatter_matrix(df_all[df_all.iloc[:,50:56].columns], diagonal='kde', figsize=(14,14));

In [1307]:
# HTGDIFF: Home Team Goal Difference
# ATGDIFF: Away Team Goal Difference

afl_df['HTGDIFF'] = afl_df['homeTeamScore.matchScore.goals'] - afl_df['awayTeamScore.matchScore.goals']
afl_df['ATGDIFF'] = afl_df['awayTeamScore.matchScore.goals'] - afl_df['homeTeamScore.matchScore.goals']

## Calculate AVG goal difference for home and away team rolling 4 Games

In [1308]:
def avg_goal_diff(df, avg_h_a_diff, a_h_team, a_h_goal_letter):
    """
    input: 
        df = dataframe with all results
        avg_h_a_diff = name of the new column
        a_h_team = HomeTeam or AwayTeam
        a_h_goal_letter = 'H' for home or 'A' for away
    output: 
        avg_per_team = dictionary with with team as key and columns as values with new column H/ATGDIFF
    """
    df[avg_h_a_diff] = 0
    avg_per_team = {}
    all_teams = df[a_h_team].unique()
    for t in all_teams:
        df_team = df[df[a_h_team]==t].fillna(0)
        result = df_team['{}TGDIFF'.format(a_h_goal_letter)].rolling(4).mean()
        df_team[avg_h_a_diff] = result
        avg_per_team[t] = df_team
    return avg_per_team

In [1309]:
d_AVGFTHG = avg_goal_diff(afl_df, 'AVGHTGDIFF', 'match.homeTeam.name', 'H')

In [1310]:
def from_dict_value_to_df(d):
    """
    input = dictionary 
    output = dataframe as part of all the values from the dictionary
    """
    df = pd.DataFrame()
    for v in d.values():
        df = pd.concat([df,v])
    return df

In [1311]:
df_AVGFTHG = from_dict_value_to_df(d_AVGFTHG)
df_AVGFTHG.sort_index(inplace=True)

In [1312]:
d_AVGFTAG = avg_goal_diff(df_AVGFTHG, 'AVGATGDIFF', 'match.awayTeam.name', 'A')
afl_df = from_dict_value_to_df(d_AVGFTAG)
afl_df.sort_index(inplace=True)
afl_df['AVGATGDIFF'].fillna(0, inplace=True)

## Add per match game results from last three games

In [1313]:
afl_df['goal_diff'] = afl_df['homeTeamScore.matchScore.goals'] - afl_df['awayTeamScore.matchScore.goals']


for index, row in df_all[df_all['match.status']=='CONCLUDED'].iterrows():
    if afl_df['goal_diff'][index] > 0:
        afl_df.at[index,'result'] = 3   # 3 is a win
    elif afl_df['goal_diff'][index] == 0:
        afl_df.at[index,'result'] = 2  # 2 is a draw
    else:
        afl_df.at[index,'result'] = 1  # 1 is a loss 

In [1314]:
afl_df.head()

,match.name,match.date,match.status,match.matchId,match.venue,match.utcStartTime,match.homeTeamId,match.awayTeamId,match.round,match.venueLocalStartTime,...,Time.on.Ground..,Substitute,group_id,date,HTGDIFF,ATGDIFF,AVGHTGDIFF,AVGATGDIFF,goal_diff,result
0,Carlton Vs Richmond,2017-03-23 08:20:00,CONCLUDED,CD_M20170140101,CD_V40,2017-03-23T08:20:00,CD_T30,CD_T120,CD_R201701401,2017-03-23T19:20:00,...,3600.0,0.0,0.0,2017-03-23 00:00:00,-6,6,0.0,0.0,-6,1.0
1,Collingwood Vs Western Bulldogs,2017-03-24 08:50:00,CONCLUDED,CD_M20170140102,CD_V40,2017-03-24T08:50:00,CD_T40,CD_T140,CD_R201701401,2017-03-24T19:50:00,...,3597.0,0.0,0.0,2017-03-24 00:00:00,-3,3,0.0,0.0,-3,1.0
2,St Kilda Vs Melbourne,2017-03-25 05:35:00,CONCLUDED,CD_M20170140103,CD_V190,2017-03-25T05:35:00,CD_T130,CD_T90,CD_R201701401,2017-03-25T16:35:00,...,3602.0,0.0,0.0,2017-03-25 00:00:00,-5,5,0.0,0.0,-5,1.0
3,Sydney Swans Vs Port Adelaide,2017-03-25 05:35:00,CONCLUDED,CD_M20170140104,CD_V60,2017-03-25T05:35:00,CD_T160,CD_T110,CD_R201701401,2017-03-25T16:35:00,...,0.0,0.0,0.0,0,-5,5,0.0,0.0,-5,1.0
4,Essendon Vs Hawthorn,2017-03-25 08:25:00,CONCLUDED,CD_M20170140106,CD_V40,2017-03-25T08:25:00,CD_T50,CD_T80,CD_R201701401,2017-03-25T19:25:00,...,3602.0,0.0,0.0,2017-03-25 00:00:00,5,-5,0.0,0.0,5,3.0


In [1315]:
def previous_data(df, h_or_a_team, column, letter, past_n):
    """
    input: 
        df = dataframe with all results
        a_h_team = HomeTeam or AwayTeam
        column = column selected to get previous data from
    output:
        team_with_past_dict = dictionary with team as a key and columns as values with new 
                              columns with past value
    """
    d = dict()
    team_with_past_dict = dict()
    all_teams = df[h_or_a_team].unique()
    for team in all_teams:
        n_games = len(df[df[h_or_a_team]==team])
        team_with_past_dict[team] = df[df[h_or_a_team]==team]
        for i in range(1, past_n):
            d[i] = team_with_past_dict[team].assign(
                result=team_with_past_dict[team].groupby(h_or_a_team)[column].shift(i)
            ).fillna({'{}_X'.format(column): 0})
            team_with_past_dict[team]['{}_{}_{}'.format(letter, column, i)] = d[i].result
    return team_with_past_dict

In [1316]:
def previous_data_call(df, side, column, letter, iterations):
    d = previous_data(df, side, column, letter, iterations)
    df_result= from_dict_value_to_df(d)
    df_result.sort_index(inplace=True)
    return df_result

In [1317]:
df_last_home_results = previous_data_call(afl_df, 'match.homeTeam.name', 'result', 'H', 3)
df_last_away_results = previous_data_call(df_last_home_results, 'match.awayTeam.name', 'result', 'A', 3)

In [1318]:
df_last_last_HTGDIFF_results = previous_data_call(df_last_away_results, 'match.homeTeam.name', 'HTGDIFF', 'H', 3)
df_last_last_ATGDIFF_results = previous_data_call(df_last_last_HTGDIFF_results, 'match.awayTeam.name', 'ATGDIFF', 'A', 3)

In [1319]:
df_last_AVGFTHG_results = previous_data_call(df_last_last_ATGDIFF_results, 'match.homeTeam.name', 'AVGHTGDIFF', 'H', 2)
df_last_AVGFTAG_results = previous_data_call(df_last_AVGFTHG_results, 'match.awayTeam.name', 'AVGATGDIFF', 'A', 2)

In [1320]:
afl_df = df_last_AVGFTAG_results.copy()

In [1321]:
afl_df.shape

(1072, 139)

In [1322]:
afl_df

,match.name,match.date,match.status,match.matchId,match.venue,match.utcStartTime,match.homeTeamId,match.awayTeamId,match.round,match.venueLocalStartTime,...,H_result_1,H_result_2,A_result_1,A_result_2,H_HTGDIFF_1,H_HTGDIFF_2,A_ATGDIFF_1,A_ATGDIFF_2,H_AVGHTGDIFF_1,A_AVGATGDIFF_1
0,Carlton Vs Richmond,2017-03-23 08:20:00,CONCLUDED,CD_M20170140101,CD_V40,2017-03-23T08:20:00,CD_T30,CD_T120,CD_R201701401,2017-03-23T19:20:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Collingwood Vs Western Bulldogs,2017-03-24 08:50:00,CONCLUDED,CD_M20170140102,CD_V40,2017-03-24T08:50:00,CD_T40,CD_T140,CD_R201701401,2017-03-24T19:50:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,St Kilda Vs Melbourne,2017-03-25 05:35:00,CONCLUDED,CD_M20170140103,CD_V190,2017-03-25T05:35:00,CD_T130,CD_T90,CD_R201701401,2017-03-25T16:35:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Sydney Swans Vs Port Adelaide,2017-03-25 05:35:00,CONCLUDED,CD_M20170140104,CD_V60,2017-03-25T05:35:00,CD_T160,CD_T110,CD_R201701401,2017-03-25T16:35:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Essendon Vs Hawthorn,2017-03-25 08:25:00,CONCLUDED,CD_M20170140106,CD_V40,2017-03-25T08:25:00,CD_T50,CD_T80,CD_R201701401,2017-03-25T19:25:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1067,Adelaide Crows Vs Brisbane Lions,2022-05-14 09:40:00,CONCLUDED,CD_M20220140906,CD_V6,2022-05-14T09:40:00,CD_T10,CD_T20,CD_R202201409,2022-05-14T19:10:00,...,NaN,NaN,NaN,NaN,-9.0,3.0,4.0,10.0,-0.75,4.75
1068,Gold Coast Suns Vs Fremantle,2022-05-15 03:40:00,CONCLUDED,CD_M20220140907,CD_V81,2022-05-15T03:40:00,CD_T1000,CD_T60,CD_R202201409,2022-05-15T13:40:00,...,NaN,NaN,NaN,NaN,-10.0,5.0,0.0,8.0,-4.25,3.75
1069,GWS Giants Vs Carlton,2022-05-15 05:20:00,CONCLUDED,CD_M20220140908,CD_V43,2022-05-15T05:20:00,CD_T1010,CD_T30,CD_R202201409,2022-05-15T15:20:00,...,NaN,NaN,NaN,NaN,-8.0,-2.0,-5.0,-5.0,-2.50,-5.75
1070,West Coast Eagles Vs Melbourne,2022-05-15 07:20:00,CONCLUDED,CD_M20220140909,CD_V2925,2022-05-15T07:20:00,CD_T150,CD_T90,CD_R202201409,2022-05-15T15:20:00,...,NaN,NaN,NaN,NaN,-17.0,-9.0,1.0,6.0,-9.50,2.25


In [1323]:
df_matches_numeric = afl_df._get_numeric_data()


In [1324]:
df_matches_numeric.columns

Index(['match.homeTeam.timeZone', 'match.awayTeam.timeZone', 'venue.capacity',
       'venue.groundDimension', 'venue.latitude', 'venue.longitude',
       'round.year', 'round.roundNumber', 'scoreWorm', 'scoreMap',
       'homeTeamScore.rushedBehinds', 'homeTeamScore.minutesInFront',
       'homeTeamScore.matchScore.totalScore', 'homeTeamScore.matchScore.goals',
       'homeTeamScore.matchScore.behinds',
       'homeTeamScore.matchScore.superGoals', 'awayTeamScore.rushedBehinds',
       'awayTeamScore.minutesInFront', 'awayTeamScore.matchScore.totalScore',
       'awayTeamScore.matchScore.goals', 'awayTeamScore.matchScore.behinds',
       'awayTeamScore.matchScore.superGoals', 'weather.tempInCelsius',
       'homeTeamScoreChart.goals', 'homeTeamScoreChart.leftBehinds',
       'homeTeamScoreChart.rightBehinds', 'homeTeamScoreChart.leftPosters',
       'homeTeamScoreChart.rightPosters', 'homeTeamScoreChart.rushedBehinds',
       'homeTeamScoreChart.touchedBehinds', 'awayTeamScoreChart.go

In [1325]:
#df_matches_numeric.drop(['match.homeTeam.timeZone', 'match.awayTeam.timeZone', 'goal_diff', 'result', 'homeTeamScore.matchScore.goals', 'awayTeamScore.matchScore.goals'], axis=1, inplace=True)


#df_matches_numeric = df_matches_numeric[['HTGDIFF','ATGDIFF','awayTeamScore.minutesInFront','homeTeamScore.minutesInFront','homeTeamScoreChart.goals','homeTeamScore.matchScore.totalScore','awayTeamScore.matchScore.totalScore','AVGHTGDIFF','round.year','awayTeamScoreChart.goals']]



df_matches_numeric.isnull().sum(axis = 0)

match.homeTeam.timeZone     0
match.awayTeam.timeZone     0
venue.capacity              0
venue.groundDimension       0
venue.latitude              0
                           ..
H_HTGDIFF_2                36
A_ATGDIFF_1                18
A_ATGDIFF_2                36
H_AVGHTGDIFF_1             18
A_AVGATGDIFF_1             18
Length: 99, dtype: int64

In [1201]:
#df_norm = (df_matches_numeric - df_matches_numeric.min()) / (df_matches_numeric.max() - df_matches_numeric.min())

In [1202]:
#df_norm.columns

In [1204]:

#

In [1326]:

all_cols = ['match.matchId','match.date', 'match.status', 'match.venue', 'match.homeTeam.name', 'match.awayTeam.name','venue.name', 'venue.state', 'round.name', 'round.year', 'round.roundNumber', 'status',
'homeTeamScore.rushedBehinds', 'homeTeamScore.minutesInFront',
       'homeTeamScore.matchScore.totalScore', 'homeTeamScore.matchScore.goals',
       'homeTeamScore.matchScore.behinds',
       'homeTeamScore.matchScore.superGoals', 'awayTeamScore.rushedBehinds',
       'awayTeamScore.minutesInFront', 'awayTeamScore.matchScore.totalScore',
       'awayTeamScore.matchScore.goals', 'awayTeamScore.matchScore.behinds',
       'awayTeamScore.matchScore.superGoals', 'weather.tempInCelsius',
       'homeTeamScoreChart.goals', 'homeTeamScoreChart.leftBehinds',
       'homeTeamScoreChart.rightBehinds', 'homeTeamScoreChart.leftPosters',
       'homeTeamScoreChart.rightPosters', 'homeTeamScoreChart.rushedBehinds',
       'homeTeamScoreChart.touchedBehinds', 'awayTeamScoreChart.goals',
       'awayTeamScoreChart.leftBehinds', 'awayTeamScoreChart.rightBehinds',
       'awayTeamScoreChart.leftPosters', 'awayTeamScoreChart.rightPosters',
       'awayTeamScoreChart.rushedBehinds', 'awayTeamScoreChart.touchedBehinds', 
       'HQ1G', 'HQ1B', 'HQ2G',
       'HQ2B', 'HQ3G', 'HQ3B', 'HQ4G', 'HQ4B', 'Home.score', 'AQ1G', 'AQ1B',
       'AQ2G', 'AQ2B', 'AQ3G', 'AQ3B', 'AQ4G', 'AQ4B', 'Away.score',
       'Kicks', 'Marks', 'Handballs', 'Goals', 'Behinds', 'Hit.Outs',
       'Tackles', 'Rebounds', 'Inside.50s', 'Clearances', 'Clangers',
       'Frees.For', 'Frees.Against', 'Brownlow.Votes', 'Contested.Possessions',
       'Uncontested.Possessions', 'Contested.Marks', 'Marks.Inside.50',
       'One.Percenters', 'Bounces', 'Goal.Assists', 'Time.on.Ground..',
       'Substitute', 'group_id', 'HTGDIFF', 'ATGDIFF', 'AVGHTGDIFF',
       'AVGATGDIFF', 'goal_diff', 'result', 'H_result_1', 'H_result_2',
       'A_result_1', 'A_result_2', 'H_HTGDIFF_1', 'H_HTGDIFF_2', 'A_ATGDIFF_1',
       'A_ATGDIFF_2', 'H_AVGHTGDIFF_1', 'A_AVGATGDIFF_1','Season']

non_feature_cols = ['match.matchId','match.date', 'match.status', 'match.venue', 'match.homeTeam.name', 'match.awayTeam.name','venue.name', 'venue.state', 'round.name', 'round.year', 'round.roundNumber', 'status','Season']
feature_cols = [
       'homeTeamScore.rushedBehinds', 'homeTeamScore.minutesInFront',
       'homeTeamScore.matchScore.totalScore', 'homeTeamScore.matchScore.goals',
       'homeTeamScore.matchScore.behinds',
       'homeTeamScore.matchScore.superGoals', 'awayTeamScore.rushedBehinds',
       'awayTeamScore.minutesInFront', 'awayTeamScore.matchScore.totalScore',
       'awayTeamScore.matchScore.goals', 'awayTeamScore.matchScore.behinds',
       'awayTeamScore.matchScore.superGoals', 'weather.tempInCelsius',
       'homeTeamScoreChart.goals', 'homeTeamScoreChart.leftBehinds',
       'homeTeamScoreChart.rightBehinds', 'homeTeamScoreChart.leftPosters',
       'homeTeamScoreChart.rightPosters', 'homeTeamScoreChart.rushedBehinds',
       'homeTeamScoreChart.touchedBehinds', 'awayTeamScoreChart.goals',
       'awayTeamScoreChart.leftBehinds', 'awayTeamScoreChart.rightBehinds',
       'awayTeamScoreChart.leftPosters', 'awayTeamScoreChart.rightPosters',
       'awayTeamScoreChart.rushedBehinds', 'awayTeamScoreChart.touchedBehinds', 
       'HQ1G', 'HQ1B', 'HQ2G',
       'HQ2B', 'HQ3G', 'HQ3B', 'HQ4G', 'HQ4B', 'Home.score', 'AQ1G', 'AQ1B',
       'AQ2G', 'AQ2B', 'AQ3G', 'AQ3B', 'AQ4G', 'AQ4B', 'Away.score',
       'Kicks', 'Marks', 'Handballs', 'Goals', 'Behinds', 'Hit.Outs',
       'Tackles', 'Rebounds', 'Inside.50s', 'Clearances', 'Clangers',
       'Frees.For', 'Frees.Against', 'Brownlow.Votes', 'Contested.Possessions',
       'Uncontested.Possessions', 'Contested.Marks', 'Marks.Inside.50',
       'One.Percenters', 'Bounces', 'Goal.Assists', 'Time.on.Ground..',
       'Substitute', 'group_id', 'HTGDIFF', 'ATGDIFF', 'AVGHTGDIFF',
       'AVGATGDIFF', 'goal_diff', 'result', 'H_result_1', 'H_result_2',
       'A_result_1', 'A_result_2', 'H_HTGDIFF_1', 'H_HTGDIFF_2', 'A_ATGDIFF_1',
       'A_ATGDIFF_2', 'H_AVGHTGDIFF_1', 'A_AVGATGDIFF_1']




In [1327]:
# Define a function which returns a DataFrame with the expontential moving average for each numeric stat
def create_exp_weighted_avgs(df, span):
    # Create a copy of the df with only the game id and the team - we will add cols to this df
    ema_features = df[['match.matchId', 'match.homeTeam.name']].copy()
    
    feature_names = [col for col in df.columns if col.startswith('f_')] # Get a list of columns we will iterate over
    
    for feature_name in feature_names:
        feature_ema = (df.groupby('match.homeTeam.name')[feature_name]
                         .transform(lambda row: (row.ewm(span=span)
                                                    .mean()
                                                    .shift(1))))
        ema_features[feature_name] = feature_ema
    
    return ema_features

In [1328]:
afl_df = afl_df[all_cols] 

afl_df = afl_df.rename(columns={col: 'f_' + col for col in afl_df if col not in non_feature_cols})



In [1329]:
afl_df

,match.matchId,match.date,match.status,match.venue,match.homeTeam.name,match.awayTeam.name,venue.name,venue.state,round.name,round.year,...,f_H_result_2,f_A_result_1,f_A_result_2,f_H_HTGDIFF_1,f_H_HTGDIFF_2,f_A_ATGDIFF_1,f_A_ATGDIFF_2,f_H_AVGHTGDIFF_1,f_A_AVGATGDIFF_1,Season
0,CD_M20170140101,2017-03-23 08:20:00,CONCLUDED,CD_V40,Carlton,Richmond,MCG,VIC,Round 1,2017,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017.0
1,CD_M20170140102,2017-03-24 08:50:00,CONCLUDED,CD_V40,Collingwood,Western Bulldogs,MCG,VIC,Round 1,2017,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017.0
2,CD_M20170140103,2017-03-25 05:35:00,CONCLUDED,CD_V190,St Kilda,Melbourne,Marvel Stadium,VIC,Round 1,2017,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017.0
3,CD_M20170140104,2017-03-25 05:35:00,CONCLUDED,CD_V60,Sydney Swans,Port Adelaide,SCG,NSW,Round 1,2017,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
4,CD_M20170140106,2017-03-25 08:25:00,CONCLUDED,CD_V40,Essendon,Hawthorn,MCG,VIC,Round 1,2017,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1067,CD_M20220140906,2022-05-14 09:40:00,CONCLUDED,CD_V6,Adelaide Crows,Brisbane Lions,Adelaide Oval,SA,Round 9,2022,...,NaN,NaN,NaN,-9.0,3.0,4.0,10.0,-0.75,4.75,0.0
1068,CD_M20220140907,2022-05-15 03:40:00,CONCLUDED,CD_V81,Gold Coast Suns,Fremantle,Metricon Stadium,QLD,Round 9,2022,...,NaN,NaN,NaN,-10.0,5.0,0.0,8.0,-4.25,3.75,0.0
1069,CD_M20220140908,2022-05-15 05:20:00,CONCLUDED,CD_V43,GWS Giants,Carlton,GIANTS Stadium,NSW,Round 9,2022,...,NaN,NaN,NaN,-8.0,-2.0,-5.0,-5.0,-2.50,-5.75,0.0
1070,CD_M20220140909,2022-05-15 07:20:00,CONCLUDED,CD_V2925,West Coast Eagles,Melbourne,Optus Stadium,WA,Round 9,2022,...,NaN,NaN,NaN,-17.0,-9.0,1.0,6.0,-9.50,2.25,0.0


In [1209]:
features_rolling_averages = create_exp_weighted_avgs(afl_df, span=10)

In [1210]:
features_rolling_averages

,match.matchId,match.homeTeam.name,f_homeTeamScore.rushedBehinds,f_homeTeamScore.minutesInFront,f_homeTeamScore.matchScore.totalScore,f_homeTeamScore.matchScore.goals,f_homeTeamScore.matchScore.behinds,f_homeTeamScore.matchScore.superGoals,f_awayTeamScore.rushedBehinds,f_awayTeamScore.minutesInFront,...,f_H_result_1,f_H_result_2,f_A_result_1,f_A_result_2,f_H_HTGDIFF_1,f_H_HTGDIFF_2,f_A_ATGDIFF_1,f_A_ATGDIFF_2,f_H_AVGHTGDIFF_1,f_A_AVGATGDIFF_1
0,CD_M20170140101,Carlton,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,CD_M20170140102,Collingwood,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,CD_M20170140103,St Kilda,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,CD_M20170140104,Sydney Swans,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,CD_M20170140106,Essendon,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
985,CD_M20210142501,Geelong Cats,1.956514,69.822090,84.264137,12.580504,8.781110,0.0,1.717857,40.707593,...,2.625644,2.542453,1.682219,1.675945,3.434343,3.753090,0.687902,2.932833,4.573635,1.174156
986,CD_M20210142502,Brisbane Lions,2.163206,91.111804,107.121318,15.697377,12.937058,0.0,2.025780,25.569536,...,2.614721,2.529102,2.150236,2.009177,5.337287,3.412209,-0.997595,-0.162466,3.973466,-1.101700
987,CD_M20210142601,Melbourne,1.938016,66.341828,80.340300,11.474148,11.495409,0.0,2.080765,41.295150,...,1.984318,1.758605,1.691276,2.134047,1.091183,-0.221919,2.220539,-0.883206,0.616011,0.554413
988,CD_M20210142602,Port Adelaide,1.773957,71.836106,92.316690,13.575835,10.861678,0.0,2.721514,37.809636,...,2.405456,2.273333,1.412603,1.431004,4.556527,2.013492,0.922223,3.608795,2.738029,1.252487


In [1211]:
features = afl_df[['match.date', 'match.matchId', 'match.homeTeam.name', 'match.awayTeam.name', 'venue.name','round.year']].copy()



In [1212]:
features = pd.merge(features, features_rolling_averages, on=['match.matchId', 'match.homeTeam.name'])

In [1213]:
#afl_df = afl_df.dropna()

In [1214]:
form_btwn_teams = afl_df[['match.matchId', 'match.homeTeam.name', 'match.awayTeam.name', 'f_goal_diff']].copy()

form_btwn_teams_inv = pd.DataFrame()

for index, row in form_btwn_teams.iterrows():
    home = row['match.homeTeam.name']
    away = row['match.awayTeam.name']
    matchid = row['match.matchId']
    margin = row['f_goal_diff']
    
    form_btwn_teams_inv = form_btwn_teams_inv.append({'match.matchId': matchid, 'match.homeTeam.name': away, 'match.awayTeam.name': home, 'f_goal_diff': -1*margin}, ignore_index=True)

    

In [1215]:
print(form_btwn_teams_inv.shape)
print(form_btwn_teams.shape)
form_btwn_teams_1 = pd.concat([form_btwn_teams, form_btwn_teams_inv], ignore_index=True, axis=0)

print(form_btwn_teams_1.shape)

(990, 4)
(990, 4)
(1980, 4)


In [1216]:
form_btwn_teams

,match.matchId,match.homeTeam.name,match.awayTeam.name,f_goal_diff
0,CD_M20170140101,Carlton,Richmond,-6
1,CD_M20170140102,Collingwood,Western Bulldogs,-3
2,CD_M20170140103,St Kilda,Melbourne,-5
3,CD_M20170140104,Sydney Swans,Port Adelaide,-5
4,CD_M20170140106,Essendon,Hawthorn,5
...,...,...,...,...
985,CD_M20210142501,Geelong Cats,GWS Giants,5
986,CD_M20210142502,Brisbane Lions,Western Bulldogs,0
987,CD_M20210142601,Melbourne,Geelong Cats,13
988,CD_M20210142602,Port Adelaide,Western Bulldogs,-11


In [1217]:
form_btwn_teams.columns

Index(['match.matchId', 'match.homeTeam.name', 'match.awayTeam.name',
       'f_goal_diff'],
      dtype='object')

In [1218]:

form_btwn_teams_1['f_form_margin_btwn_teams'] = (form_btwn_teams_1.groupby(['match.homeTeam.name', 'match.awayTeam.name'])['f_goal_diff']
                                                          .transform(lambda row: row.rolling(5).mean().shift())
                                                          .fillna(0))

form_btwn_teams_1['f_form_past_5_btwn_teams'] = \
(form_btwn_teams_1.assign(win=lambda df: df.apply(lambda row: 1 if row.f_goal_diff > 0 else 0, axis='columns'))
              .groupby(['match.homeTeam.name', 'match.awayTeam.name'])['win']
              .transform(lambda row: row.rolling(5).mean().shift() * 5)
              .fillna(0))

In [1219]:
form_btwn_teams_1

,match.matchId,match.homeTeam.name,match.awayTeam.name,f_goal_diff,f_form_margin_btwn_teams,f_form_past_5_btwn_teams
0,CD_M20170140101,Carlton,Richmond,-6,0.0,0.0
1,CD_M20170140102,Collingwood,Western Bulldogs,-3,0.0,0.0
2,CD_M20170140103,St Kilda,Melbourne,-5,0.0,0.0
3,CD_M20170140104,Sydney Swans,Port Adelaide,-5,0.0,0.0
4,CD_M20170140106,Essendon,Hawthorn,5,0.0,0.0
...,...,...,...,...,...,...
1975,CD_M20210142501,GWS Giants,Geelong Cats,-5,-1.2,2.0
1976,CD_M20210142502,Western Bulldogs,Brisbane Lions,0,-0.2,3.0
1977,CD_M20210142601,Geelong Cats,Melbourne,-13,-1.0,2.0
1978,CD_M20210142602,Western Bulldogs,Port Adelaide,11,-0.6,3.0


In [1220]:
print(features.shape)

(990, 91)


In [1221]:
features

,match.date,match.matchId,match.homeTeam.name,match.awayTeam.name,venue.name,round.year,f_homeTeamScore.rushedBehinds,f_homeTeamScore.minutesInFront,f_homeTeamScore.matchScore.totalScore,f_homeTeamScore.matchScore.goals,...,f_H_result_1,f_H_result_2,f_A_result_1,f_A_result_2,f_H_HTGDIFF_1,f_H_HTGDIFF_2,f_A_ATGDIFF_1,f_A_ATGDIFF_2,f_H_AVGHTGDIFF_1,f_A_AVGATGDIFF_1
0,2017-03-23 08:20:00,CD_M20170140101,Carlton,Richmond,MCG,2017,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2017-03-24 08:50:00,CD_M20170140102,Collingwood,Western Bulldogs,MCG,2017,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2017-03-25 05:35:00,CD_M20170140103,St Kilda,Melbourne,Marvel Stadium,2017,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2017-03-25 05:35:00,CD_M20170140104,Sydney Swans,Port Adelaide,SCG,2017,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2017-03-25 08:25:00,CD_M20170140106,Essendon,Hawthorn,MCG,2017,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
985,2021-09-03 09:50:00,CD_M20210142501,Geelong Cats,GWS Giants,Optus Stadium,2021,1.956514,69.822090,84.264137,12.580504,...,2.625644,2.542453,1.682219,1.675945,3.434343,3.753090,0.687902,2.932833,4.573635,1.174156
986,2021-09-04 09:20:00,CD_M20210142502,Brisbane Lions,Western Bulldogs,Gabba,2021,2.163206,91.111804,107.121318,15.697377,...,2.614721,2.529102,2.150236,2.009177,5.337287,3.412209,-0.997595,-0.162466,3.973466,-1.101700
987,2021-09-10 09:50:00,CD_M20210142601,Melbourne,Geelong Cats,Optus Stadium,2021,1.938016,66.341828,80.340300,11.474148,...,1.984318,1.758605,1.691276,2.134047,1.091183,-0.221919,2.220539,-0.883206,0.616011,0.554413
988,2021-09-11 09:40:00,CD_M20210142602,Port Adelaide,Western Bulldogs,Adelaide Oval,2021,1.773957,71.836106,92.316690,13.575835,...,2.405456,2.273333,1.412603,1.431004,4.556527,2.013492,0.922223,3.608795,2.738029,1.252487


In [1222]:
#print(features.shape)
# Merge to our features df
#features = pd.merge(features, form_btwn_teams_1.drop(columns=['f_goal_diff']), on=['match.matchId', 'match.homeTeam.name', 'match.awayTeam.name'])

#print(features.shape)

In [1223]:
features

,match.date,match.matchId,match.homeTeam.name,match.awayTeam.name,venue.name,round.year,f_homeTeamScore.rushedBehinds,f_homeTeamScore.minutesInFront,f_homeTeamScore.matchScore.totalScore,f_homeTeamScore.matchScore.goals,...,f_H_result_1,f_H_result_2,f_A_result_1,f_A_result_2,f_H_HTGDIFF_1,f_H_HTGDIFF_2,f_A_ATGDIFF_1,f_A_ATGDIFF_2,f_H_AVGHTGDIFF_1,f_A_AVGATGDIFF_1
0,2017-03-23 08:20:00,CD_M20170140101,Carlton,Richmond,MCG,2017,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2017-03-24 08:50:00,CD_M20170140102,Collingwood,Western Bulldogs,MCG,2017,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2017-03-25 05:35:00,CD_M20170140103,St Kilda,Melbourne,Marvel Stadium,2017,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2017-03-25 05:35:00,CD_M20170140104,Sydney Swans,Port Adelaide,SCG,2017,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2017-03-25 08:25:00,CD_M20170140106,Essendon,Hawthorn,MCG,2017,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
985,2021-09-03 09:50:00,CD_M20210142501,Geelong Cats,GWS Giants,Optus Stadium,2021,1.956514,69.822090,84.264137,12.580504,...,2.625644,2.542453,1.682219,1.675945,3.434343,3.753090,0.687902,2.932833,4.573635,1.174156
986,2021-09-04 09:20:00,CD_M20210142502,Brisbane Lions,Western Bulldogs,Gabba,2021,2.163206,91.111804,107.121318,15.697377,...,2.614721,2.529102,2.150236,2.009177,5.337287,3.412209,-0.997595,-0.162466,3.973466,-1.101700
987,2021-09-10 09:50:00,CD_M20210142601,Melbourne,Geelong Cats,Optus Stadium,2021,1.938016,66.341828,80.340300,11.474148,...,1.984318,1.758605,1.691276,2.134047,1.091183,-0.221919,2.220539,-0.883206,0.616011,0.554413
988,2021-09-11 09:40:00,CD_M20210142602,Port Adelaide,Western Bulldogs,Adelaide Oval,2021,1.773957,71.836106,92.316690,13.575835,...,2.405456,2.273333,1.412603,1.431004,4.556527,2.013492,0.922223,3.608795,2.738029,1.252487


In [1224]:
features.columns

Index(['match.date', 'match.matchId', 'match.homeTeam.name',
       'match.awayTeam.name', 'venue.name', 'round.year',
       'f_homeTeamScore.rushedBehinds', 'f_homeTeamScore.minutesInFront',
       'f_homeTeamScore.matchScore.totalScore',
       'f_homeTeamScore.matchScore.goals',
       'f_homeTeamScore.matchScore.behinds',
       'f_homeTeamScore.matchScore.superGoals',
       'f_awayTeamScore.rushedBehinds', 'f_awayTeamScore.minutesInFront',
       'f_awayTeamScore.matchScore.totalScore',
       'f_awayTeamScore.matchScore.goals',
       'f_awayTeamScore.matchScore.behinds',
       'f_awayTeamScore.matchScore.superGoals', 'f_weather.tempInCelsius',
       'f_homeTeamScoreChart.goals', 'f_homeTeamScoreChart.leftBehinds',
       'f_homeTeamScoreChart.rightBehinds', 'f_homeTeamScoreChart.leftPosters',
       'f_homeTeamScoreChart.rightPosters',
       'f_homeTeamScoreChart.rushedBehinds',
       'f_homeTeamScoreChart.touchedBehinds', 'f_awayTeamScoreChart.goals',
       'f_awayTeamSc

In [1225]:
# Get the result and merge to the feature_df

match_results = (afl_df.assign(result=lambda df: df.apply(lambda row: 1 if row['f_goal_diff'] > 0 else 0, axis=1)))


# Merge result column to feature_df
feature_df = pd.merge(features, match_results[['match.matchId', 'result']], on='match.matchId')


In [1226]:
afl_df = afl_df.dropna()
feature_df = feature_df.dropna()

In [1227]:

# Create our test and train sets from our afl DataFrame; drop the columns which leak the result, duplicates, and the advanced
# stats which don't have data until 2015

fnote



In [1228]:
X

,f_homeTeamScore.rushedBehinds,f_homeTeamScore.minutesInFront,f_homeTeamScore.matchScore.totalScore,f_homeTeamScore.matchScore.goals,f_homeTeamScore.matchScore.behinds,f_homeTeamScore.matchScore.superGoals,f_awayTeamScore.rushedBehinds,f_awayTeamScore.minutesInFront,f_awayTeamScore.matchScore.totalScore,f_awayTeamScore.matchScore.goals,...,f_H_result_1,f_H_result_2,f_A_result_1,f_A_result_2,f_H_HTGDIFF_1,f_H_HTGDIFF_2,f_A_ATGDIFF_1,f_A_ATGDIFF_2,f_H_AVGHTGDIFF_1,f_A_AVGATGDIFF_1
46,0.909751,0.273376,0.385800,0.409151,0.028128,0.0,-0.954473,0.190279,1.896865,1.852548,...,-2.660216,-2.545018,-0.080666,-3.039609,-3.208049,-1.204282,1.223506,3.700141,-0.287410,0.660061
49,-0.826447,-0.778517,-0.258623,-0.333825,0.390628,0.0,1.751761,1.059386,2.162480,1.901970,...,-2.660216,-2.545018,2.698888,-3.039609,-2.333033,-2.772106,-1.937555,1.597284,-0.287410,0.660061
54,1.246139,0.292352,0.756202,0.517879,2.008661,0.0,0.064421,-0.016054,2.141368,2.233577,...,-0.068429,-2.545018,-3.477898,-3.039609,-0.469574,-1.831412,3.945530,3.700141,-0.287410,0.660061
55,-1.114235,-0.603234,0.386157,0.383140,0.209621,0.0,1.195858,0.767355,1.816167,1.679269,...,-1.713051,-2.545018,0.215861,-0.052421,-1.505768,-2.254724,-0.128536,-2.797687,-0.287410,0.660061
58,0.470276,-1.007280,1.630812,1.792965,-0.317379,0.0,1.777720,1.421926,1.946944,1.986466,...,-0.068429,-2.545018,-0.698344,-3.039609,2.074454,-0.577153,0.103963,1.597284,-0.287410,0.660061
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
778,0.768059,0.874655,-1.076971,-0.911099,-1.667349,0.0,-0.694668,-1.769890,-2.493192,-2.550665,...,1.590392,1.440834,-1.669182,0.779315,1.146861,1.069593,1.169028,-0.512620,1.289710,0.222728
779,-0.559030,0.024351,-0.688414,-0.485674,-1.730828,0.0,0.088845,-0.805591,-2.009795,-2.103063,...,1.087582,0.850294,0.738086,0.020989,1.621414,1.142960,-0.250374,0.132876,1.314628,0.219891
780,-0.567159,0.728694,-1.028602,-0.988565,-0.778408,0.0,-0.884693,-1.484362,-2.147702,-2.085309,...,1.117572,0.885502,-0.046390,0.447372,0.700853,0.402743,-0.690773,-0.334894,0.569607,-0.054942
781,-0.065128,1.002803,-0.770761,-0.853378,0.191116,0.0,0.400584,-1.552454,-1.718204,-1.518561,...,1.182300,0.961533,-0.287053,-1.692397,0.485084,0.565746,0.637895,0.659505,0.527996,0.443946


In [1229]:
# Create a list of standard classifiers
classifiers = [
    #Ensemble Methods
    ensemble.AdaBoostClassifier(),
    ensemble.BaggingClassifier(),
    ensemble.ExtraTreesClassifier(),
    ensemble.GradientBoostingClassifier(),
    ensemble.RandomForestClassifier(),

    #Gaussian Processes
    gaussian_process.GaussianProcessClassifier(),
    
    #GLM
    linear_model.LogisticRegressionCV(),
    
    #Navies Bayes
    naive_bayes.BernoulliNB(),
    naive_bayes.GaussianNB(),
    
    #SVM
    svm.SVC(probability=True),
    svm.NuSVC(probability=True),
    
    #Discriminant Analysis
    discriminant_analysis.LinearDiscriminantAnalysis(),
    discriminant_analysis.QuadraticDiscriminantAnalysis(),

    
    #xgboost: http://xgboost.readthedocs.io/en/latest/model.html
#    XGBClassifier()    
]

# Define a functiom which finds the best algorithms for our modelling task
def find_best_algorithms(classifier_list, X, y):
    # This function is adapted from https://www.kaggle.com/yassineghouzam/titanic-top-4-with-ensemble-modeling
    # Cross validate model with Kfold stratified cross validation
    kfold = StratifiedKFold(n_splits=5)
    
    # Grab the cross validation scores for each algorithm
    cv_results = [cross_val_score(classifier, X, y, scoring = "neg_log_loss", cv = kfold) for classifier in classifier_list]
    cv_means = [cv_result.mean() * -1 for cv_result in cv_results]
    cv_std = [cv_result.std() for cv_result in cv_results]
    algorithm_names = [alg.__class__.__name__ for alg in classifiers]
    
    # Create a DataFrame of all the CV results
    cv_results = pd.DataFrame({
        "Mean Log Loss": cv_means,
        "Log Loss Std": cv_std,
        "Algorithm": algorithm_names
    })
    
    
    return cv_results.sort_values(by='Mean Log Loss').reset_index(drop=True)

In [1230]:
best_algos = find_best_algorithms(classifiers, X, y)
best_algos

,Mean Log Loss,Log Loss Std,Algorithm
0,0.674294,0.017877,LogisticRegressionCV
1,0.675111,0.015423,ExtraTreesClassifier
2,0.675180,0.018263,RandomForestClassifier
3,0.676045,0.010382,SVC
4,0.692753,0.019674,NuSVC
5,0.692991,0.000200,GaussianProcessClassifier
6,0.698273,0.007510,AdaBoostClassifier
7,0.726459,0.022666,GradientBoostingClassifier
8,0.851785,0.245988,LinearDiscriminantAnalysis
9,0.872860,0.164989,BaggingClassifier


In [1231]:
# Try a logistic regression model and see how it performs in terms of accuracy
kfold = StratifiedKFold(n_splits=5)
cv_scores = cross_val_score(linear_model.LogisticRegressionCV(), X, y, scoring='accuracy', cv=kfold)
cv_scores.mean()

0.5909872461029758

In [1232]:
chosen_algorithms = best_algos.loc[best_algos['Mean Log Loss'] < 0.68, 'Algorithm'].tolist()
chosen_algorithms

['LogisticRegressionCV',
 'ExtraTreesClassifier',
 'RandomForestClassifier',
 'SVC']

In [1233]:
# Define a function which optimises the hyperparameters of our chosen algorithms
def optimise_hyperparameters(train_x, train_y, algorithms, parameters):
    kfold = StratifiedKFold(n_splits=5)
    best_estimators = []
    
    for alg, params in zip(algorithms, parameters):
        gs = GridSearchCV(alg, param_grid=params, cv=kfold, scoring='neg_log_loss', verbose=1)
        gs.fit(train_x, train_y)
        best_estimators.append(gs.best_estimator_)
    return best_estimators

# Define our parameters to run a grid search over
lr_grid = {
    "C": [0.0001, 0.001, 0.01, 0.05, 0.2, 0.5],
    "solver": ["newton-cg", "lbfgs", "liblinear"]
}

# Add our algorithms and parameters to lists to be used in our function
alg_list = [LogisticRegression()]
param_list = [lr_grid]

In [1234]:
# Find the best estimators, then add our other estimators which don't need optimisation
best_estimators = optimise_hyperparameters(X, y, alg_list, param_list)

Fitting 5 folds for each of 18 candidates, totalling 90 fits


In [1235]:
lr_best_params = best_estimators[0].get_params()
lr_best_params

{'C': 0.01,
 'class_weight': None,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'l1_ratio': None,
 'max_iter': 100,
 'multi_class': 'auto',
 'n_jobs': None,
 'penalty': 'l2',
 'random_state': None,
 'solver': 'liblinear',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False}

In [1236]:
kfold = StratifiedKFold(n_splits=10)
cv_scores = cross_val_score(linear_model.LogisticRegression(**lr_best_params), X, y, scoring='neg_log_loss', cv=kfold)
cv_scores.mean()

-0.671366618722019

In [1237]:
lr = LogisticRegression(**lr_best_params)
lr.fit(X, y)
final_predictions = lr.predict(test_x)

accuracy = (final_predictions == test_y).mean() * 100

print("Our accuracy in predicting the 2018 season is: {:.2f}%".format(accuracy))

Our accuracy in predicting the 2018 season is: 55.56%


In [1238]:
predictions_probs = lr.predict_proba(test_x)

In [1239]:
metrics.log_loss(test_y, predictions_probs)

0.6631660203478413

In [1240]:
df_next_games_teams.head()

,match.homeTeam.name,match.awayTeam.name
90,Sydney Swans,Richmond
91,Brisbane Lions,GWS Giants
92,Geelong Cats,Adelaide Crows
93,Melbourne,Fremantle
94,West Coast Eagles,Western Bulldogs


In [1241]:
df_fixture.columns

# venue.name

Index(['id', 'providerId', 'utcStartTime', 'status', 'compSeason.id',
       'compSeason.providerId', 'compSeason.name', 'compSeason.shortName',
       'compSeason.currentRoundNumber', 'round.id', 'round.providerId',
       'round.abbreviation', 'round.name', 'round.roundNumber', 'round.byes',
       'home.team.id', 'home.team.providerId', 'home.team.name',
       'home.team.abbreviation', 'home.team.nickname', 'home.team.teamType',
       'home.team.club.id', 'home.team.club.providerId', 'home.team.club.name',
       'home.team.club.abbreviation', 'home.team.club.nickname',
       'home.score.goals', 'home.score.behinds', 'home.score.totalScore',
       'home.score.superGoals', 'away.team.id', 'away.team.providerId',
       'away.team.name', 'away.team.abbreviation', 'away.team.nickname',
       'away.team.teamType', 'away.team.club.id', 'away.team.club.providerId',
       'away.team.club.name', 'away.team.club.abbreviation',
       'away.team.club.nickname', 'away.score.goals', 'away

In [1242]:
df_next_games_teams.columns

Index(['match.homeTeam.name', 'match.awayTeam.name'], dtype='object')

In [1243]:
df_next_games_teams = df_fixture[(df_fixture['status'] != "CONCLUDED") & (df_fixture['round.roundNumber'] == 11)]
df_next_games_teams = df_next_games_teams[['home.team.name','away.team.name','venue.name','compSeason.year','compSeason.currentRoundNumber']]
df_next_games_teams = df_next_games_teams.rename(columns={'home.team.name': 'match.homeTeam.name', 'away.team.name': 'match.awayTeam.name','compSeason.year':'round.year','compSeason.currentRoundNumber':'round.roundNumber'})
df_next_games_teams

,match.homeTeam.name,match.awayTeam.name,venue.name,round.year,round.roundNumber
90,Sydney Swans,Richmond,SCG,2022,10
91,Brisbane Lions,GWS Giants,Gabba,2022,10
92,Geelong Cats,Adelaide Crows,GMHBA Stadium,2022,10
93,Melbourne,Fremantle,MCG,2022,10
94,West Coast Eagles,Western Bulldogs,Optus Stadium,2022,10
95,Gold Coast Suns,Hawthorn,TIO Stadium,2022,10
96,St Kilda,North Melbourne,Marvel Stadium,2022,10
97,Collingwood,Carlton,MCG,2022,10
98,Port Adelaide,Essendon,Adelaide Oval,2022,10


In [1244]:
afl_df[['match.homeTeam.name','match.awayTeam.name','venue.name','round.year','match.date']]


,match.homeTeam.name,match.awayTeam.name,venue.name,round.year,match.date
28,North Melbourne,Western Bulldogs,Marvel Stadium,2017,2017-04-14 06:20:00
38,Gold Coast Suns,Adelaide Crows,Metricon Stadium,2017,2017-04-22 06:35:00
39,Sydney Swans,GWS Giants,SCG,2017,2017-04-22 09:25:00
41,St Kilda,Geelong Cats,Marvel Stadium,2017,2017-04-23 05:20:00
42,Hawthorn,West Coast Eagles,MCG,2017,2017-04-23 06:40:00
...,...,...,...,...,...
985,Geelong Cats,GWS Giants,Optus Stadium,2021,2021-09-03 09:50:00
986,Brisbane Lions,Western Bulldogs,Gabba,2021,2021-09-04 09:20:00
987,Melbourne,Geelong Cats,Optus Stadium,2021,2021-09-10 09:50:00
988,Port Adelaide,Western Bulldogs,Adelaide Oval,2021,2021-09-11 09:40:00


In [1245]:


afl_data = afl_df.append(df_next_games_teams).reset_index(drop=True)





In [1246]:
afl_data.tail()

,match.matchId,match.date,match.status,match.venue,match.homeTeam.name,match.awayTeam.name,venue.name,venue.state,round.name,round.year,...,f_H_result_2,f_A_result_1,f_A_result_2,f_H_HTGDIFF_1,f_H_HTGDIFF_2,f_A_ATGDIFF_1,f_A_ATGDIFF_2,f_H_AVGHTGDIFF_1,f_A_AVGATGDIFF_1,Season
955,NaN,NaN,NaN,NaN,West Coast Eagles,Western Bulldogs,Optus Stadium,NaN,NaN,2022,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
956,NaN,NaN,NaN,NaN,Gold Coast Suns,Hawthorn,TIO Stadium,NaN,NaN,2022,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
957,NaN,NaN,NaN,NaN,St Kilda,North Melbourne,Marvel Stadium,NaN,NaN,2022,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
958,NaN,NaN,NaN,NaN,Collingwood,Carlton,MCG,NaN,NaN,2022,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
959,NaN,NaN,NaN,NaN,Port Adelaide,Essendon,Adelaide Oval,NaN,NaN,2022,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [1247]:
features_rolling_averages = create_exp_weighted_avgs(afl_data, span=10)

#features = pd.merge(features, features_rolling_averages, on=['game', 'team'])

In [1248]:
features_rolling_averages.tail()

,match.matchId,match.homeTeam.name,f_homeTeamScore.rushedBehinds,f_homeTeamScore.minutesInFront,f_homeTeamScore.matchScore.totalScore,f_homeTeamScore.matchScore.goals,f_homeTeamScore.matchScore.behinds,f_homeTeamScore.matchScore.superGoals,f_awayTeamScore.rushedBehinds,f_awayTeamScore.minutesInFront,...,f_H_result_1,f_H_result_2,f_A_result_1,f_A_result_2,f_H_HTGDIFF_1,f_H_HTGDIFF_2,f_A_ATGDIFF_1,f_A_ATGDIFF_2,f_H_AVGHTGDIFF_1,f_A_AVGATGDIFF_1
955,NaN,West Coast Eagles,1.678850,58.593949,77.708592,11.448560,9.017233,0.0,1.692148,55.197563,...,2.239899,2.070988,1.797974,1.727270,0.803361,0.759675,3.028950,0.398025,1.542379,0.904616
956,NaN,Gold Coast Suns,2.169249,36.789995,57.180926,7.921465,9.652136,0.0,2.997215,71.284376,...,1.540075,1.660092,1.734427,1.996513,-3.643422,-1.341875,1.202213,-1.152658,-1.787120,-0.349131
957,NaN,St Kilda,2.072744,71.867696,82.811113,12.119953,10.091396,0.0,1.975409,34.822257,...,1.903380,1.659670,1.936918,1.445606,-0.438276,-1.424631,-0.500125,3.605006,-0.581163,-0.527044
958,NaN,Collingwood,1.912048,52.700545,68.034627,9.833914,9.031142,0.0,1.492472,49.088527,...,1.271635,1.331999,1.840508,2.450866,-2.051196,-1.618122,0.232497,-3.308486,-1.657993,-2.133147
959,NaN,Port Adelaide,1.633241,58.774636,83.713305,12.198357,10.523163,0.0,2.590339,52.935506,...,2.513559,2.405454,1.519401,1.716280,5.000801,4.556499,0.754553,2.225355,3.285683,1.161131


In [1249]:
features

,match.date,match.matchId,match.homeTeam.name,match.awayTeam.name,venue.name,round.year,f_homeTeamScore.rushedBehinds,f_homeTeamScore.minutesInFront,f_homeTeamScore.matchScore.totalScore,f_homeTeamScore.matchScore.goals,...,f_H_result_1,f_H_result_2,f_A_result_1,f_A_result_2,f_H_HTGDIFF_1,f_H_HTGDIFF_2,f_A_ATGDIFF_1,f_A_ATGDIFF_2,f_H_AVGHTGDIFF_1,f_A_AVGATGDIFF_1
0,2017-03-23 08:20:00,CD_M20170140101,Carlton,Richmond,MCG,2017,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2017-03-24 08:50:00,CD_M20170140102,Collingwood,Western Bulldogs,MCG,2017,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2017-03-25 05:35:00,CD_M20170140103,St Kilda,Melbourne,Marvel Stadium,2017,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2017-03-25 05:35:00,CD_M20170140104,Sydney Swans,Port Adelaide,SCG,2017,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2017-03-25 08:25:00,CD_M20170140106,Essendon,Hawthorn,MCG,2017,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
985,2021-09-03 09:50:00,CD_M20210142501,Geelong Cats,GWS Giants,Optus Stadium,2021,1.956514,69.822090,84.264137,12.580504,...,2.625644,2.542453,1.682219,1.675945,3.434343,3.753090,0.687902,2.932833,4.573635,1.174156
986,2021-09-04 09:20:00,CD_M20210142502,Brisbane Lions,Western Bulldogs,Gabba,2021,2.163206,91.111804,107.121318,15.697377,...,2.614721,2.529102,2.150236,2.009177,5.337287,3.412209,-0.997595,-0.162466,3.973466,-1.101700
987,2021-09-10 09:50:00,CD_M20210142601,Melbourne,Geelong Cats,Optus Stadium,2021,1.938016,66.341828,80.340300,11.474148,...,1.984318,1.758605,1.691276,2.134047,1.091183,-0.221919,2.220539,-0.883206,0.616011,0.554413
988,2021-09-11 09:40:00,CD_M20210142602,Port Adelaide,Western Bulldogs,Adelaide Oval,2021,1.773957,71.836106,92.316690,13.575835,...,2.405456,2.273333,1.412603,1.431004,4.556527,2.013492,0.922223,3.608795,2.738029,1.252487


In [1250]:
features = afl_data[['match.date', 'match.matchId', 'match.homeTeam.name', 'match.awayTeam.name', 'venue.name','round.year']].copy()


In [1251]:
features.tail()

,match.date,match.matchId,match.homeTeam.name,match.awayTeam.name,venue.name,round.year
955,NaN,NaN,West Coast Eagles,Western Bulldogs,Optus Stadium,2022
956,NaN,NaN,Gold Coast Suns,Hawthorn,TIO Stadium,2022
957,NaN,NaN,St Kilda,North Melbourne,Marvel Stadium,2022
958,NaN,NaN,Collingwood,Carlton,MCG,2022
959,NaN,NaN,Port Adelaide,Essendon,Adelaide Oval,2022


In [1252]:
df_next_games_teams

,match.homeTeam.name,match.awayTeam.name,venue.name,round.year,round.roundNumber
90,Sydney Swans,Richmond,SCG,2022,10
91,Brisbane Lions,GWS Giants,Gabba,2022,10
92,Geelong Cats,Adelaide Crows,GMHBA Stadium,2022,10
93,Melbourne,Fremantle,MCG,2022,10
94,West Coast Eagles,Western Bulldogs,Optus Stadium,2022,10
95,Gold Coast Suns,Hawthorn,TIO Stadium,2022,10
96,St Kilda,North Melbourne,Marvel Stadium,2022,10
97,Collingwood,Carlton,MCG,2022,10
98,Port Adelaide,Essendon,Adelaide Oval,2022,10


In [1253]:
features.tail()

,match.date,match.matchId,match.homeTeam.name,match.awayTeam.name,venue.name,round.year
955,NaN,NaN,West Coast Eagles,Western Bulldogs,Optus Stadium,2022
956,NaN,NaN,Gold Coast Suns,Hawthorn,TIO Stadium,2022
957,NaN,NaN,St Kilda,North Melbourne,Marvel Stadium,2022
958,NaN,NaN,Collingwood,Carlton,MCG,2022
959,NaN,NaN,Port Adelaide,Essendon,Adelaide Oval,2022


In [1254]:
new_features = features.tail(9)

In [1255]:
new_features

,match.date,match.matchId,match.homeTeam.name,match.awayTeam.name,venue.name,round.year
951,NaN,NaN,Sydney Swans,Richmond,SCG,2022
952,NaN,NaN,Brisbane Lions,GWS Giants,Gabba,2022
953,NaN,NaN,Geelong Cats,Adelaide Crows,GMHBA Stadium,2022
954,NaN,NaN,Melbourne,Fremantle,MCG,2022
955,NaN,NaN,West Coast Eagles,Western Bulldogs,Optus Stadium,2022
956,NaN,NaN,Gold Coast Suns,Hawthorn,TIO Stadium,2022
957,NaN,NaN,St Kilda,North Melbourne,Marvel Stadium,2022
958,NaN,NaN,Collingwood,Carlton,MCG,2022
959,NaN,NaN,Port Adelaide,Essendon,Adelaide Oval,2022


In [1256]:
new_features_roll_av = features_rolling_averages.tail(9)

In [1257]:
new_features_roll_av

,match.matchId,match.homeTeam.name,f_homeTeamScore.rushedBehinds,f_homeTeamScore.minutesInFront,f_homeTeamScore.matchScore.totalScore,f_homeTeamScore.matchScore.goals,f_homeTeamScore.matchScore.behinds,f_homeTeamScore.matchScore.superGoals,f_awayTeamScore.rushedBehinds,f_awayTeamScore.minutesInFront,...,f_H_result_1,f_H_result_2,f_A_result_1,f_A_result_2,f_H_HTGDIFF_1,f_H_HTGDIFF_2,f_A_ATGDIFF_1,f_A_ATGDIFF_2,f_H_AVGHTGDIFF_1,f_A_AVGATGDIFF_1
951,NaN,Sydney Swans,1.824570,62.703847,91.655945,13.421269,11.128328,0.0,1.704400,46.603798,...,2.612225,2.526042,1.656147,2.135488,5.662569,3.587478,0.302562,-1.059365,3.174366,-1.370983
952,NaN,Brisbane Lions,2.133537,85.455396,101.826604,14.843318,12.766698,0.0,2.021085,28.920242,...,2.684777,2.614721,2.304743,1.825688,5.457806,5.337287,-1.543511,0.594356,4.205575,-0.446840
953,NaN,Geelong Cats,2.146241,76.218231,87.670568,13.020396,9.548193,0.0,1.769161,33.487842,...,2.511891,2.625644,1.558179,1.553045,2.809917,3.434343,0.744639,2.581405,3.923885,1.369768
954,NaN,Melbourne,2.437856,77.567854,97.831309,14.325702,11.877099,0.0,2.351593,33.941261,...,2.320083,2.168990,1.760273,1.940971,3.689172,1.620067,2.445205,-0.293707,1.544619,1.280240
955,NaN,West Coast Eagles,1.678850,58.593949,77.708592,11.448560,9.017233,0.0,1.692148,55.197563,...,2.239899,2.070988,1.797974,1.727270,0.803361,0.759675,3.028950,0.398025,1.542379,0.904616
956,NaN,Gold Coast Suns,2.169249,36.789995,57.180926,7.921465,9.652136,0.0,2.997215,71.284376,...,1.540075,1.660092,1.734427,1.996513,-3.643422,-1.341875,1.202213,-1.152658,-1.787120,-0.349131
957,NaN,St Kilda,2.072744,71.867696,82.811113,12.119953,10.091396,0.0,1.975409,34.822257,...,1.903380,1.659670,1.936918,1.445606,-0.438276,-1.424631,-0.500125,3.605006,-0.581163,-0.527044
958,NaN,Collingwood,1.912048,52.700545,68.034627,9.833914,9.031142,0.0,1.492472,49.088527,...,1.271635,1.331999,1.840508,2.450866,-2.051196,-1.618122,0.232497,-3.308486,-1.657993,-2.133147
959,NaN,Port Adelaide,1.633241,58.774636,83.713305,12.198357,10.523163,0.0,2.590339,52.935506,...,2.513559,2.405454,1.519401,1.716280,5.000801,4.556499,0.754553,2.225355,3.285683,1.161131


In [1258]:
next_round_x = pd.merge(new_features, new_features_roll_av, on=['match.homeTeam.name'])

In [1259]:
next_round_x

,match.date,match.matchId_x,match.homeTeam.name,match.awayTeam.name,venue.name,round.year,match.matchId_y,f_homeTeamScore.rushedBehinds,f_homeTeamScore.minutesInFront,f_homeTeamScore.matchScore.totalScore,...,f_H_result_1,f_H_result_2,f_A_result_1,f_A_result_2,f_H_HTGDIFF_1,f_H_HTGDIFF_2,f_A_ATGDIFF_1,f_A_ATGDIFF_2,f_H_AVGHTGDIFF_1,f_A_AVGATGDIFF_1
0,NaN,NaN,Sydney Swans,Richmond,SCG,2022,NaN,1.824570,62.703847,91.655945,...,2.612225,2.526042,1.656147,2.135488,5.662569,3.587478,0.302562,-1.059365,3.174366,-1.370983
1,NaN,NaN,Brisbane Lions,GWS Giants,Gabba,2022,NaN,2.133537,85.455396,101.826604,...,2.684777,2.614721,2.304743,1.825688,5.457806,5.337287,-1.543511,0.594356,4.205575,-0.446840
2,NaN,NaN,Geelong Cats,Adelaide Crows,GMHBA Stadium,2022,NaN,2.146241,76.218231,87.670568,...,2.511891,2.625644,1.558179,1.553045,2.809917,3.434343,0.744639,2.581405,3.923885,1.369768
3,NaN,NaN,Melbourne,Fremantle,MCG,2022,NaN,2.437856,77.567854,97.831309,...,2.320083,2.168990,1.760273,1.940971,3.689172,1.620067,2.445205,-0.293707,1.544619,1.280240
4,NaN,NaN,West Coast Eagles,Western Bulldogs,Optus Stadium,2022,NaN,1.678850,58.593949,77.708592,...,2.239899,2.070988,1.797974,1.727270,0.803361,0.759675,3.028950,0.398025,1.542379,0.904616
5,NaN,NaN,Gold Coast Suns,Hawthorn,TIO Stadium,2022,NaN,2.169249,36.789995,57.180926,...,1.540075,1.660092,1.734427,1.996513,-3.643422,-1.341875,1.202213,-1.152658,-1.787120,-0.349131
6,NaN,NaN,St Kilda,North Melbourne,Marvel Stadium,2022,NaN,2.072744,71.867696,82.811113,...,1.903380,1.659670,1.936918,1.445606,-0.438276,-1.424631,-0.500125,3.605006,-0.581163,-0.527044
7,NaN,NaN,Collingwood,Carlton,MCG,2022,NaN,1.912048,52.700545,68.034627,...,1.271635,1.331999,1.840508,2.450866,-2.051196,-1.618122,0.232497,-3.308486,-1.657993,-2.133147
8,NaN,NaN,Port Adelaide,Essendon,Adelaide Oval,2022,NaN,1.633241,58.774636,83.713305,...,2.513559,2.405454,1.519401,1.716280,5.000801,4.556499,0.754553,2.225355,3.285683,1.161131


In [1260]:
feature_cols = [col for col in afl_data if col.startswith('f_')]


In [1262]:
next_round_x

,match.date,match.matchId_x,match.homeTeam.name,match.awayTeam.name,venue.name,round.year,match.matchId_y,f_homeTeamScore.rushedBehinds,f_homeTeamScore.minutesInFront,f_homeTeamScore.matchScore.totalScore,...,f_H_result_1,f_H_result_2,f_A_result_1,f_A_result_2,f_H_HTGDIFF_1,f_H_HTGDIFF_2,f_A_ATGDIFF_1,f_A_ATGDIFF_2,f_H_AVGHTGDIFF_1,f_A_AVGATGDIFF_1
0,NaN,NaN,Sydney Swans,Richmond,SCG,2022,NaN,1.824570,62.703847,91.655945,...,2.612225,2.526042,1.656147,2.135488,5.662569,3.587478,0.302562,-1.059365,3.174366,-1.370983
1,NaN,NaN,Brisbane Lions,GWS Giants,Gabba,2022,NaN,2.133537,85.455396,101.826604,...,2.684777,2.614721,2.304743,1.825688,5.457806,5.337287,-1.543511,0.594356,4.205575,-0.446840
2,NaN,NaN,Geelong Cats,Adelaide Crows,GMHBA Stadium,2022,NaN,2.146241,76.218231,87.670568,...,2.511891,2.625644,1.558179,1.553045,2.809917,3.434343,0.744639,2.581405,3.923885,1.369768
3,NaN,NaN,Melbourne,Fremantle,MCG,2022,NaN,2.437856,77.567854,97.831309,...,2.320083,2.168990,1.760273,1.940971,3.689172,1.620067,2.445205,-0.293707,1.544619,1.280240
4,NaN,NaN,West Coast Eagles,Western Bulldogs,Optus Stadium,2022,NaN,1.678850,58.593949,77.708592,...,2.239899,2.070988,1.797974,1.727270,0.803361,0.759675,3.028950,0.398025,1.542379,0.904616
5,NaN,NaN,Gold Coast Suns,Hawthorn,TIO Stadium,2022,NaN,2.169249,36.789995,57.180926,...,1.540075,1.660092,1.734427,1.996513,-3.643422,-1.341875,1.202213,-1.152658,-1.787120,-0.349131
6,NaN,NaN,St Kilda,North Melbourne,Marvel Stadium,2022,NaN,2.072744,71.867696,82.811113,...,1.903380,1.659670,1.936918,1.445606,-0.438276,-1.424631,-0.500125,3.605006,-0.581163,-0.527044
7,NaN,NaN,Collingwood,Carlton,MCG,2022,NaN,1.912048,52.700545,68.034627,...,1.271635,1.331999,1.840508,2.450866,-2.051196,-1.618122,0.232497,-3.308486,-1.657993,-2.133147
8,NaN,NaN,Port Adelaide,Essendon,Adelaide Oval,2022,NaN,1.633241,58.774636,83.713305,...,2.513559,2.405454,1.519401,1.716280,5.000801,4.556499,0.754553,2.225355,3.285683,1.161131


In [1274]:
for i in range(0,len(prediction_probs)):
    print(prediction_probs[i].round(3))

[0.1 0.9]
[1. 0.]
[0.082 0.918]
[1. 0.]
[0.143 0.857]
[0.263 0.737]
[1. 0.]
[1. 0.]
[1. 0.]


In [1273]:
df_next_games_teams

,match.homeTeam.name,match.awayTeam.name,venue.name,round.year,round.roundNumber
90,Sydney Swans,Richmond,SCG,2022,10
91,Brisbane Lions,GWS Giants,Gabba,2022,10
92,Geelong Cats,Adelaide Crows,GMHBA Stadium,2022,10
93,Melbourne,Fremantle,MCG,2022,10
94,West Coast Eagles,Western Bulldogs,Optus Stadium,2022,10
95,Gold Coast Suns,Hawthorn,TIO Stadium,2022,10
96,St Kilda,North Melbourne,Marvel Stadium,2022,10
97,Collingwood,Carlton,MCG,2022,10
98,Port Adelaide,Essendon,Adelaide Oval,2022,10


In [ ]:
feature_cols = [col for col in train_df if col.startswith('f_')]

# Scale features
scaler = StandardScaler()
train_x[feature_cols] = scaler.fit_transform(train_x[feature_cols])
next_round_x[feature_cols] = scaler.transform(next_round_x[feature_cols])

lr = LogisticRegression(**lr_best_params)
lr.fit(train_x[feature_cols], train_y)
prediction_probs = lr.predict_proba(next_round_x[feature_cols])

,match.date,match.matchId,match.homeTeam.name,match.awayTeam.name,venue.name,round.year
0,2017-04-14 06:20:00,CD_M20170140402,North Melbourne,Western Bulldogs,Marvel Stadium,2017
1,2017-04-22 06:35:00,CD_M20170140503,Gold Coast Suns,Adelaide Crows,Metricon Stadium,2017
2,2017-04-22 09:25:00,CD_M20170140504,Sydney Swans,GWS Giants,SCG,2017
3,2017-04-23 05:20:00,CD_M20170140506,St Kilda,Geelong Cats,Marvel Stadium,2017
4,2017-04-23 06:40:00,CD_M20170140507,Hawthorn,West Coast Eagles,MCG,2017
...,...,...,...,...,...,...
955,NaN,NaN,West Coast Eagles,Western Bulldogs,Optus Stadium,2022
956,NaN,NaN,Gold Coast Suns,Hawthorn,TIO Stadium,2022
957,NaN,NaN,St Kilda,North Melbourne,Marvel Stadium,2022
958,NaN,NaN,Collingwood,Carlton,MCG,2022
